Grab info from /Users/localadmin/Documents/projects/QM_ESP_Psi4/examples/prop_test_2.db database



In [37]:
import pickle
import sys
import os 
import numpy as np
import asyncio


sys.path.append('/Users/localadmin/Documents/projects/QM_ESP_Psi4')

from source.storage.storage import MoleculePropRecord, MoleculePropStore

from openff.recharge.esp import ESPSettings
from openff.recharge.grids import MSKGridSettings
from openff.recharge.esp.psi4 import Psi4ESPGenerator
from openff.toolkit.topology import Molecule
from openff.recharge.utilities.toolkits import VdWRadiiType, compute_vdw_radii
from openff.units import unit
from openff.utilities import temporary_cd

from molesp.models import ESPMolecule, Surface
from molesp.gui import launch
from molesp.cli._cli import compute_surface


In [10]:
prop_store = MoleculePropStore("/Users/localadmin/Documents/projects/QM_ESP_Psi4/examples/prop_test_2.db")

In [11]:
smiles_list = prop_store.list()
smiles_list

['OCC(O)CO',
 'C#CC',
 'C1CN1',
 'C1COC1',
 'CC#N',
 'CC(=O)[O-]',
 'CC(C)=O',
 'CCCC',
 'CCNCC',
 'CN(C)C',
 'CN=[N+]=[N-]',
 'CNC',
 'COC',
 'CSC',
 'Fc1ccccc1',
 'NCO',
 'Nc1ccccc1',
 'O=[NH+][O-]',
 'Oc1ccccc1',
 'CCl',
 'CF',
 'CO',
 'CS',
 'C1COCO1',
 'c1ccccc1',
 'c1ccncc1',
 'c1ccsc1']

In [12]:
test_mol = prop_store.retrieve(smiles_list[0])[0]
tagged_smiles = test_mol.tagged_smiles
openff_molecule = Molecule.from_mapped_smiles(tagged_smiles)
test_mol.conformer_quantity

Magnitude,[[-1.7448371652633583 -1.1692305991533931 0.35291475127711647] [-1.300437452867219 0.16068228144279137 0.3880603045529657] [0.05525721478904798 0.3467314756964777 -0.29181887031399634] [0.38554207385941075 1.7094352724281747 -0.32620771923321035] [1.1616603978720228 -0.3800404430573445 0.4562040882231809] [2.3738648179096633 -0.4039067229678673 -0.24407074021139039] [-2.0399941537247335 -1.3860815110521711 -0.5202961821136074] [-1.222584713967644 0.4377925591363111 1.431303663930806] [-2.0286469236178277 0.8286420734799702 -0.06890978969169474] [-0.0014278273316579873 -0.05678343089204132 -1.3059272421231995] [-0.13019718433876246 2.1555339381577814 -0.9829449299349872] [1.3530864329570143 0.14943412961138441 1.3802505096767455] [0.8290122883627434 -1.3837833925477012 0.7011821925438567] [2.309702195361457 -1.0084256302824248 -0.9697400365825318]]
Units,angstrom


In [13]:
vdw_radii = compute_vdw_radii(openff_molecule, radii_type=VdWRadiiType.Bondi)

radii = (
            np.array([[radii] for radii in vdw_radii.m_as(unit.angstrom)])
            * unit.angstrom
        )

In [17]:
vertices, indices = compute_surface(molecule = openff_molecule,
                                    conformer = test_mol.conformer_quantity, 
                                    radii = radii,
                                    radii_scale = 1.4,
                                    spacing = 0.2 * unit.angstrom )

In [41]:
esp_settings = ESPSettings(
            basis="6-31G*", method="hf", grid_settings=MSKGridSettings()
        )

with temporary_cd():

    _, esp, _ = Psi4ESPGenerator._generate(
        openff_molecule,
        test_mol.conformer_quantity,
        vertices * unit.angstrom,
        esp_settings,
        "",
        minimize=False,
        compute_esp=True,
        compute_field=False,
    )


In [31]:
esp_molecule =  ESPMolecule(
    atomic_numbers = [atom.atomic_number for atom in openff_molecule.atoms],
    conformer = test_mol.conformer.flatten().tolist(),
    surface = Surface(
            vertices=vertices.flatten().tolist(), indices=indices.flatten().tolist()
        ),
        esp={"QC ESP": esp.m_as(unit.hartree / unit.e).flatten().tolist()},
    )

In [40]:
async def your_coroutine():
    launch(esp_molecule, port=port)

loop = asyncio.get_event_loop()
loop.run_until_complete(your_coroutine())

/Users/localadmin/mambaforge/envs/molesp/lib/python3.10/ast.py:50: RuntimeWarning: coroutine 'Server.serve' was never awaited
  return compile(source, filename, mode, flags,


RuntimeError: This event loop is already running

In [42]:
from esp_from_conformer import MyESPProcessor

The GUI will be available at http://localhost:8000 after a few seconds.

RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
MyESP